In [19]:
import os
print("Current directory:", os.getcwd())
print("Git branch:", os.popen('git branch --show-current').read().strip())

Current directory: /home/ileniag/buzi_ml4cad_noncvd
Git branch: noncvd-7y-alive


In [20]:
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
from hyperparameters import hyperparameters
import matplotlib as mpl

from IPython.display import display
plt.style.use('bmh')
mpl.rcParams.update({
    "grid.linestyle" : "dashed",
    "axes.facecolor" : "white",
    "axes.spines.top" : False,
    "axes.spines.right" : False,
    "legend.frameon" : False,
    "figure.figsize" : (8, 5),
    "figure.dpi" : 300,
})
%matplotlib inline

# Suppress sklearn deprecated warnings
import warnings
def warn(*args, **kwargs): pass
warnings.warn = warn
np.set_printoptions(threshold=sys.maxsize)

np.random.seed(42)

In [21]:
# TODO make test for dataset with creatina column
# Dataset without thyroid = 18 features (including survive7y)
# Dataset with thyroid = 27 features (including survive7y)
# With columns that have missing values, 23 and 32
# Default 18
n_features = 18
extra_path = n_features != 27 and n_features != 18
dropped_na_key = "dropped_na/"
mean_key = "mean/"
key = mean_key
path = f"data/{n_features}features/{key if extra_path else '' }"
path_models = f"models/{n_features}features/{key if extra_path else '' }"
output_models = f"models_output/{n_features}features/{key if extra_path else '' }"
print(path_models)
print(path)
print(output_models)

models/18features/
data/18features/
models_output/18features/


In [22]:
# Read data
df_train = pd.read_csv(f"{path}train.csv", index_col=0)
df_valid = pd.read_csv(f"{path}valid.csv", index_col=0)
df_test = pd.read_csv(f"{path}test.csv", index_col=0)
print(len(df_train) + len(df_valid) + len(df_test))
print(len(df_train.columns))


train, valid, test = df_train.to_numpy(), df_valid.to_numpy(), df_test.to_numpy()

# y_**** contains the value of Survive7y as a list
# X_**** contains everything except for Survive7y as a list of list
X_train, y_train = train[:, :-1], train[:, -1]
X_valid, y_valid = valid[:, :-1], valid[:, -1]
X_test, y_test = test[:, :-1], test[:, -1]
feat_names = list(df_train.columns)
# Print how Survive7y are distribuited in each set
from collections import Counter
print(Counter(y_train))
print(Counter(y_valid))
print(Counter(y_test))

# All the numerical features that can be standardized
from utils import get_preprocess_std_num
preprocess_std = get_preprocess_std_num(feat_names)

# Preprocessed ready-to-use train and valid set
process_tmp = preprocess_std.fit(X_train)
X_train_std = process_tmp.transform(X_train)
X_valid_std = process_tmp.transform(X_valid)

#If you want to print the resulting df
# Note: You don't need to pass the _std to the train function. The function will call predict on the pipeline and transform the dataset accordingly to the transformer  
#df_scaled = pd.DataFrame(X_valid_std,columns = preprocess_std.get_feature_names_out())
#display(df_scaled)

7018
18
Counter({np.float64(1.0): 3705, np.float64(0.0): 505})
Counter({np.float64(1.0): 1235, np.float64(0.0): 169})
Counter({np.float64(1.0): 1235, np.float64(0.0): 169})


### Training


In [23]:
from functools import partial
from train import report, evaluate, train_and_evaluate
train_partial = partial(
    train_and_evaluate, 
    preprocess_std, 
    X_train=X_train,
    y_train=y_train,
    X_valid=X_valid,
    y_valid=y_valid,
    scoring="f1_macro", 
    iter=5000, 
    save=True,
    path_models = path_models,
    output_models = output_models
)

In [24]:
from sklearn.linear_model import LogisticRegression

hyperparams = hyperparameters["lr"] 
#Default is None (thus weight = 1). Balanced uses the formula n_samples / (n_classes * np.bincount(y))
model = LogisticRegression(class_weight="balanced")
train_partial(model=model, hyperparams=hyperparams, savename="lr")

/home/ileniag/buzi_ml4cad_noncvd/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ileniag/buzi_ml4cad_noncvd/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ileniag/buzi_ml4cad_noncvd/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ileniag/buzi_ml4cad_noncvd/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ileniag/buzi_ml4cad_noncvd/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ileniag/b

Testing on training set:
              precision    recall  f1-score   support

         0.0      0.295     0.721     0.418       505
         1.0      0.953     0.765     0.849      3705

    accuracy                          0.760      4210
   macro avg      0.624     0.743     0.633      4210
weighted avg      0.874     0.760     0.797      4210

auc macro 0.814
confusion matrix
[[ 364  141]
 [ 871 2834]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.302     0.710     0.424       169
         1.0      0.951     0.776     0.855      1235

    accuracy                          0.768      1404
   macro avg      0.627     0.743     0.639      1404
weighted avg      0.873     0.768     0.803      1404

auc macro 0.820
confusion matrix
[[120  49]
 [277 958]]
Model rank: 1
Mean validation score: 0.641 (std: 0.005)
Parameters: {'model__C': 7, 'model__dual': True, 'model__max_iter': 70, 'model__penalty': 'l2', 'model__solver': 'liblinea

,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('stand', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [25]:
from sklearn.svm import SVC
hyperparams = hyperparameters["svc"] 

model = SVC(class_weight="balanced", probability=True)
train_partial(model=model, hyperparams=hyperparams, savename="svc")

/home/ileniag/buzi_ml4cad_noncvd/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1600).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/ileniag/buzi_ml4cad_noncvd/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1600).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/ileniag/buzi_ml4cad_noncvd/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=800).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/ileniag/buzi_ml4cad_noncvd/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=800).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/ileniag/buzi_ml4cad_

Testing on training set:
              precision    recall  f1-score   support

         0.0      0.194     0.453     0.271       505
         1.0      0.909     0.743     0.817      3705

    accuracy                          0.708      4210
   macro avg      0.551     0.598     0.544      4210
weighted avg      0.823     0.708     0.752      4210

auc macro 0.633
confusion matrix
[[ 229  276]
 [ 953 2752]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.211     0.485     0.294       169
         1.0      0.914     0.752     0.825      1235

    accuracy                          0.720      1404
   macro avg      0.563     0.619     0.560      1404
weighted avg      0.830     0.720     0.761      1404

auc macro 0.648
confusion matrix
[[ 82  87]
 [306 929]]
Model rank: 1
Mean validation score: 0.631 (std: 0.014)
Parameters: {'model__C': 260, 'model__coef0': np.float64(0.4764319555781835), 'model__degree': 72, 'model__gamma': 'auto',

,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('stand', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [26]:
from sklearn.neighbors import KNeighborsClassifier

hyperparams = hyperparameters["knn"] 

model = KNeighborsClassifier()
train_partial(model=model, hyperparams=hyperparams, savename="knn")

Testing on training set:
              precision    recall  f1-score   support

         0.0      0.634     0.481     0.547       505
         1.0      0.932     0.962     0.947      3705

    accuracy                          0.905      4210
   macro avg      0.783     0.722     0.747      4210
weighted avg      0.896     0.905     0.899      4210

auc macro 0.925
confusion matrix
[[ 243  262]
 [ 140 3565]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.405     0.278     0.330       169
         1.0      0.905     0.944     0.924      1235

    accuracy                          0.864      1404
   macro avg      0.655     0.611     0.627      1404
weighted avg      0.845     0.864     0.853      1404

auc macro 0.718
confusion matrix
[[  47  122]
 [  69 1166]]
Model rank: 1
Mean validation score: 0.603 (std: 0.004)
Parameters: {'model__algorithm': 'ball_tree', 'model__leaf_size': 38, 'model__n_neighbors': 4, 'model__weights': 'unif

,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('stand', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [27]:
from sklearn.ensemble import RandomForestClassifier

hyperparams = hyperparameters["rf"] 

model = RandomForestClassifier()
train_partial(model=model, hyperparams=hyperparams, savename="rf")

Testing on training set:
              precision    recall  f1-score   support

         0.0      0.588     0.869     0.701       505
         1.0      0.981     0.917     0.948      3705

    accuracy                          0.911      4210
   macro avg      0.784     0.893     0.825      4210
weighted avg      0.934     0.911     0.918      4210

auc macro 0.968
confusion matrix
[[ 439   66]
 [ 308 3397]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.413     0.550     0.472       169
         1.0      0.936     0.893     0.914      1235

    accuracy                          0.852      1404
   macro avg      0.674     0.722     0.693      1404
weighted avg      0.873     0.852     0.861      1404

auc macro 0.828
confusion matrix
[[  93   76]
 [ 132 1103]]
Model rank: 1
Mean validation score: 0.666 (std: 0.003)
Parameters: {'model__class_weight': 'balanced', 'model__criterion': 'gini', 'model__max_features': 'log2', 'model__min

,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('stand', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [28]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

hyperparams = hyperparameters["adaboost"] 

model = AdaBoostClassifier()
train_partial(model=model, hyperparams=hyperparams, savename="adaboost")

Testing on training set:
              precision    recall  f1-score   support

         0.0      0.620     0.194     0.296       505
         1.0      0.900     0.984     0.940      3705

    accuracy                          0.889      4210
   macro avg      0.760     0.589     0.618      4210
weighted avg      0.866     0.889     0.863      4210

auc macro 0.821
confusion matrix
[[  98  407]
 [  60 3645]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.614     0.207     0.310       169
         1.0      0.901     0.982     0.940      1235

    accuracy                          0.889      1404
   macro avg      0.757     0.595     0.625      1404
weighted avg      0.866     0.889     0.864      1404

auc macro 0.823
confusion matrix
[[  35  134]
 [  22 1213]]
Model rank: 1
Mean validation score: 0.615 (std: 0.001)
Parameters: {'model__learning_rate': np.float64(1.0756954125989546), 'model__n_estimators': 89}

Model rank: 2
Mean va

,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('stand', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [29]:
from sklearn.neural_network import MLPClassifier
import random

hyperparams = hyperparameters["nn"] 

model = MLPClassifier()
train_partial(model=model, hyperparams=hyperparams, savename="nn")

Testing on training set:
              precision    recall  f1-score   support

         0.0      0.684     0.206     0.317       505
         1.0      0.901     0.987     0.942      3705

    accuracy                          0.893      4210
   macro avg      0.793     0.596     0.629      4210
weighted avg      0.875     0.893     0.867      4210

auc macro 0.825
confusion matrix
[[ 104  401]
 [  48 3657]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.702     0.237     0.354       169
         1.0      0.904     0.986     0.943      1235

    accuracy                          0.896      1404
   macro avg      0.803     0.611     0.649      1404
weighted avg      0.880     0.896     0.872      1404

auc macro 0.818
confusion matrix
[[  40  129]
 [  17 1218]]
Model rank: 1
Mean validation score: 0.650 (std: 0.003)
Parameters: {'model__alpha': np.float64(0.1022495542616435), 'model__early_stopping': True, 'model__hidden_layer_sizes

,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('stand', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [30]:
from sklearn.ensemble import GradientBoostingClassifier

hyperparams = hyperparameters["gb"] 

model = GradientBoostingClassifier()
train_partial(model=model, hyperparams=hyperparams, savename="gb")

Testing on training set:
              precision    recall  f1-score   support

         0.0      0.704     0.311     0.431       505
         1.0      0.913     0.982     0.946      3705

    accuracy                          0.902      4210
   macro avg      0.808     0.647     0.689      4210
weighted avg      0.888     0.902     0.884      4210

auc macro 0.864
confusion matrix
[[ 157  348]
 [  66 3639]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.584     0.266     0.366       169
         1.0      0.907     0.974     0.939      1235

    accuracy                          0.889      1404
   macro avg      0.745     0.620     0.652      1404
weighted avg      0.868     0.889     0.870      1404

auc macro 0.810
confusion matrix
[[  45  124]
 [  32 1203]]
Model rank: 1
Mean validation score: 0.638 (std: 0.014)
Parameters: {'model__learning_rate': np.float64(0.16252886807948705), 'model__max_depth': 5, 'model__max_features': 'l

,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('stand', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [31]:
#Don't run this in jupyter within vscode, run this with notebooks within browsers.
import os
#os.environ['KMP_DUPLICATE_LIB_OK']='True'

import xgboost as xgb

hyperparams = hyperparameters["xgb"] 

model = xgb.XGBClassifier(n_jobs=1)
train_partial(model=model, hyperparams=hyperparams, savename="xgb")

/home/ileniag/buzi_ml4cad_noncvd/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [00:26:11] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "gamma", "max_depth", "subsample" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/ileniag/buzi_ml4cad_noncvd/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [00:26:11] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "gamma", "max_depth", "subsample" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/ileniag/buzi_ml4cad_noncvd/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [00:26:11] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "gamma", "max_depth", "subsample" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/ileniag/buzi_ml4cad_noncvd/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [00:26:11] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "gamma", "max_depth", "subs

Testing on training set:
              precision    recall  f1-score   support

         0.0      0.486     0.404     0.441       505
         1.0      0.921     0.942     0.931      3705

    accuracy                          0.877      4210
   macro avg      0.703     0.673     0.686      4210
weighted avg      0.868     0.877     0.872      4210

auc macro 0.835
confusion matrix
[[ 204  301]
 [ 216 3489]]
Testing on validation set:
              precision    recall  f1-score   support

         0.0      0.532     0.438     0.481       169
         1.0      0.925     0.947     0.936      1235

    accuracy                          0.886      1404
   macro avg      0.729     0.693     0.708      1404
weighted avg      0.878     0.886     0.881      1404

auc macro 0.826
confusion matrix
[[  74   95]
 [  65 1170]]
Model rank: 1
Mean validation score: 0.671 (std: 0.007)
Parameters: {'model__alpha': np.float64(0.011795231881217005), 'model__booster': 'dart', 'model__eta': np.float64(0.09

,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('stand', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [32]:

mean_path = f"data/27features/"
df_train = pd.read_csv(f"{mean_path}train.csv")
df_valid = pd.read_csv(f"{mean_path}valid.csv")
df_test = pd.read_csv(f"{mean_path}test.csv")
sum_valid = 0
sum_test = 0
print(1 in df_valid.iloc[:,0].to_numpy())
for val in df_train.iloc[:,0].to_numpy():
    if val in df_valid.iloc[:,0].to_numpy():
        print(val)
        sum_valid +=1
    if val in df_test.iloc[:,0].to_numpy():
        sum_test +=1
print("#######################")
print(sum_valid)
print(sum_test)

False
#######################
0
0


In [ ]:
import pandas as pd
from sklearn.metrics import classification_report, roc_auc_score
import joblib

model_names_map = {
    'lr': 'Logistic Regression',
    'svc': 'Support Vector Machine',
    'knn': 'K-Nearest Neighbors',
    'rf': 'Random Forest',
    'adaboost': 'AdaBoost',
    'nn': 'Neural Network (MLP)',
    'gb': 'Gradient Boosting',
    'xgb': 'XGBoost'
}

results_summary = []

for model_key in ['lr', 'svc', 'knn', 'rf', 'adaboost', 'nn', 'gb', 'xgb']:
    model_name = model_names_map[model_key]
    model_path = f"{path_models}{model_key}.joblib"
    model = joblib.load(model_path)

    # VALIDATION set
    y_pred = model.predict(X_valid)
    y_proba = model.predict_proba(X_valid)

    report = classification_report(y_valid, y_pred, output_dict=True)

    # ✨ qui prendiamo la riga "macro avg"
    precision_macro = report['macro avg']['precision']
    recall_macro = report['macro avg']['recall']
    f1_macro = report['macro avg']['f1-score']

    # AUC "macro": nel binario è la stessa dell'AUC standard
    auc_macro = roc_auc_score(y_valid, y_proba[:, 1])

    results_summary.append({
        'Model': model_name,
        'Precision (macro)': f"{precision_macro:.3f}",
        'Recall (macro)': f"{recall_macro:.3f}",
        'F1-Score (macro)': f"{f1_macro:.3f}",
        'AUC (macro)': f"{auc_macro:.3f}",
    })

df_summary = pd.DataFrame(results_summary)

print(f"\n{'='*90}")
print(f"SUMMARY TABLE - VALIDATION SET ({n_features} features)")
print(f"{'='*90}\n")
print(df_summary.to_string(index=False))
print(f"\n{'='*90}\n")

output_path = f"figures/{n_features}features/models_summary_validation.csv"
df_summary.to_csv(output_path, index=False)
print(f"Table saved to: {output_path}")

# best by macro
best_f1_macro_idx = df_summary['F1-Score (macro)'].astype(float).idxmax()
best_auc_idx = df_summary['AUC (macro)'].astype(float).idxmax()
best_recall_idx = df_summary['Recall (macro)'].astype(float).idxmax()

print("\nBEST MODELS BY METRIC:")
print(f"  - Best F1-Score (macro): {df_summary.loc[best_f1_macro_idx, 'Model']} "
      f"({df_summary.loc[best_f1_macro_idx, 'F1-Score (macro)']})")
print(f"  - Best AUC (macro): {df_summary.loc[best_auc_idx, 'Model']} "
      f"({df_summary.loc[best_auc_idx, 'AUC (macro)']})")
print(f"  - Best Recall (macro): {df_summary.loc[best_recall_idx, 'Model']} "
      f"({df_summary.loc[best_recall_idx, 'Recall (macro)']})")



SUMMARY TABLE - VALIDATION SET (18 features)

                 Model Precision Recall F1-Score F1-Score (macro avg)   AUC
   Logistic Regression     0.302  0.710    0.424                0.639 0.820
Support Vector Machine     0.211  0.485    0.294                0.560 0.648
   K-Nearest Neighbors     0.405  0.278    0.330                0.627 0.718
         Random Forest     0.413  0.550    0.472                0.693 0.828
              AdaBoost     0.614  0.207    0.310                0.625 0.823
  Neural Network (MLP)     0.702  0.237    0.354                0.649 0.818
     Gradient Boosting     0.584  0.266    0.366                0.652 0.810
               XGBoost     0.532  0.438    0.481                0.708 0.826


Table saved to: figures/18features/models_summary_validation.csv

BEST MODELS BY METRIC:
  - Best F1-Score (macro avg): XGBoost (0.708)
  - Best AUC: Random Forest (0.828)
  - Best Recall (class 0 - CVD deaths): Logistic Regression (0.710)


In [34]:
from auto_export_notebook import export_current_notebook


html_path = export_current_notebook(
    globals(),
    wait_for_disk_save=True,   # wait for Auto Save
    wait_timeout_sec=8.0
)
print("Exported to:", html_path)


<IPython.core.display.Javascript object>

Exported to: /home/ileniag/buzi_ml4cad_noncvd/exported_notebooks/2_classifiers_18features_20251028_003054.html
